In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import warnings
import gc
import time

time_start=time.time()

In [2]:
answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])

In [3]:
def caul_time(x):
    time=x[1:].split('-H')
    return int(time[0])*24+int(time[1])

In [24]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
train=train[['qid','uid','time']]
#test=test[['qid','uid','time']]
all_data=pd.concat([train,test])

In [5]:
from tqdm import tqdm 
tqdm.pandas()

In [6]:
answer_info['atime']=answer_info['atime'].progress_apply(lambda x:caul_time(x))

100%|██████████| 4513735/4513735 [00:11<00:00, 384159.87it/s]


In [7]:
answer_info=answer_info[['qid','uid','atime']]

In [8]:
all_data['time']=all_data['time'].progress_apply(lambda x:caul_time(x))

100%|██████████| 10630880/10630880 [00:27<00:00, 389865.84it/s]


In [9]:
len(all_data)

10630880

In [10]:
all_data=pd.merge(all_data,answer_info,on='uid',how='inner')
all_data['sub_time']=all_data['time']-all_data['atime']
all_data=all_data[all_data['sub_time']>0]
len(all_data)


92201588

In [11]:
all_data=all_data[all_data['qid_x']!=all_data['qid_y']]
len(all_data)

92201567

In [12]:
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid_x','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
ques_info=ques_info[['qid_x','qtitlec']]
all_data=pd.merge(all_data,ques_info,on='qid_x',how='left')#141527246
ques_info.columns=['qid_y','qtitlec_h']
all_data=pd.merge(all_data,ques_info,on='qid_y',how='left')
len(all_data)

92201567

In [13]:
all_data

,qid_x,uid,time,qid_y,atime,sub_time,qtitlec,qtitlec_h
0,Q604029601,M2317670257,92703,Q3034061280,92364,339,"W31489,W9218,W3440,W243","W272,W263,W4534,W7742"
1,Q604029601,M2317670257,92703,Q1309562090,92539,164,"W31489,W9218,W3440,W243","W1116,W6761,W17922"
2,Q426016802,M2317670257,92653,Q3034061280,92364,289,"W272,W6761","W272,W263,W4534,W7742"
3,Q426016802,M2317670257,92653,Q1309562090,92539,114,"W272,W6761","W1116,W6761,W17922"
4,Q4274079416,M2317670257,92532,Q3034061280,92364,168,"W4961,W9343,W4707,W22173,W162804,W16271,W10013...","W272,W263,W4534,W7742"
5,Q3093602909,M2317670257,92559,Q3034061280,92364,195,"W20352,W34999,W216697,W2072,W10789,W5597","W272,W263,W4534,W7742"
6,Q3093602909,M2317670257,92559,Q1309562090,92539,20,"W20352,W34999,W216697,W2072,W10789,W5597","W1116,W6761,W17922"
7,Q1416840932,M2317670257,92485,Q3034061280,92364,121,"W575779,W1909,W7742","W272,W263,W4534,W7742"
8,Q1216060994,M2317670257,92600,Q3034061280,92364,236,"W53331,W22173,W1234487,W1365,W614","W272,W263,W4534,W7742"
9,Q1216060994,M2317670257,92600,Q1309562090,92539,61,"W53331,W22173,W1234487,W1365,W614","W1116,W6761,W17922"


In [14]:
all_data=all_data[all_data['qtitlec']!=all_data['qtitlec_h']]
len(all_data)

92197494

In [15]:
import math
import pandas as pd
import numpy as np
import gc
class bm25_Scorer(object):
    def __init__(self, docs):
        self.docs = docs
        self.D = len(self.docs)
        self.avgdl = sum([len(doc) for doc in self.docs]) / self.D
        self.df = {}
        self.idf = {}
        self.k1 = 1.5
        self.b = 0.75
        self.initialize()

    def initialize(self, ngram=1):
        for doc in self.docs:
            word_set = set(doc)
            for word in word_set:
                if word not in self.df:
                    self.df[word] = 1
                self.df[word] += 1
        for k, v in self.df.items():
            self.idf[k] = math.log(self.D - v + 0.5) - math.log(v + 0.5)
    def sim(self, doc1, doc2):
        score = 0
        doc1=doc1.split(',')
        doc2=doc2.split(',')
        d = len(doc2)
        loc = self.build_local_df(doc2)
        for word in doc1:
            if word not in loc:
                continue
            score += (self.idf[word] * loc[word] * (self.k1 + 1)
                      / (loc[word] + self.k1 * (1 - self.b + self.b * d
                                                / self.avgdl)))
        return score

    def build_local_df(self, doc):
        tmp = {}
        for word in doc:
            if not word in tmp:
                tmp[word] = 0
            tmp[word] += 1
        return tmp

    def add_ngram(self, n):
        idx = 0
        for doc in self.docs:
            ngram = self.generate_ngram(n, self.titles[idx])
            seg_list = seg_list + ngram
            idx += 1

    def generate_ngram(self, n, sentence):
        return [sentence[i:i + n] for i in range(0, len(sentence) - 1)]

In [16]:
ques_info['qtitlec_h']=ques_info['qtitlec_h'].progress_apply(lambda x:x.split(','))

100%|██████████| 1829900/1829900 [00:05<00:00, 333700.79it/s]


In [17]:
docs=ques_info['qtitlec_h'].tolist()
bm25Scorer = bm25_Scorer(docs=docs)


In [18]:
all_data['bm25_q1_to_q2'] = all_data[['qtitlec', 'qtitlec_h']].progress_apply(lambda row: bm25Scorer.sim(row['qtitlec'], row['qtitlec_h']), axis=1)
all_data['bm25_q2_to_q1'] = all_data[['qtitlec', 'qtitlec_h']].progress_apply(lambda row: bm25Scorer.sim(row['qtitlec_h'], row['qtitlec']), axis=1)

100%|██████████| 92197494/92197494 [52:12<00:00, 29430.71it/s] 
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
100%|██████████| 92197494/92197494 [51:26<00:00, 29867.36it/s] 
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
all_data=all_data.merge(feature,on=['qid','uid'],how='left')

In [19]:
feature=all_data[['qid_x','uid','bm25_q1_to_q2','bm25_q2_to_q1']].groupby(['qid_x','uid']).agg('mean')

In [20]:
feature=feature.reset_index()

In [29]:
feature

,qid,uid,bm25_12_mean,bm25_21_mean
0,Q1000002524,M2358184554,0.000000,0.000000
1,Q1000002524,M562088453,0.000000,0.000000
2,Q1000006560,M1628216851,2.796815,2.882277
3,Q1000007604,M2627631122,0.000000,0.000000
4,Q1000007604,M274784249,0.000000,0.000000
5,Q1000007604,M3737115870,6.258868,5.802323
6,Q1000007604,M973577865,0.000000,0.000000
7,Q1000008276,M1455895037,5.692014,6.249993
8,Q1000008276,M1501011241,4.048635,4.048635
9,Q1000008276,M1755649682,3.743598,4.225676


In [26]:
feature.columns=['qid','uid','bm25_12_mean','bm25_21_mean']

In [30]:
feature.to_hdf('feature_qtitle_bm25new.h5', key='data')